# Aprofundando o RAG: Base de Conhecimento e Banco Vetorial

## ✅ 1. Revisão Rápida do RAG com LangChain

O que é um RAG?
RAG = Retrieval-Augmented Generation

Ele busca informações relevantes antes de gerar a resposta.

Usa LLM + banco de vetores.

Como funciona na prática:
O usuário envia uma pergunta.

A pergunta é vetorizada.

O sistema busca vetores semelhantes na base.

As informações encontradas são repassadas para a LLM, que responde com contexto.

Componentes principais:
Embeddings (vetorização): convertem texto em vetores numéricos.

Vector Store (FAISS): armazena e busca os vetores.

Retriever + LLM: fazem a mágica acontecer.



In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.docstore.document import Document

# Base de conhecimento
documentos = [
    Document(page_content="Nosso suporte funciona das 9h às 18h, de segunda a sexta."),
    Document(page_content="O prazo de reembolso é de até 7 dias úteis."),
]

# Vetorização
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(documentos, embeddings)


## 📌 2. Vetorizando Documentos Reais

### Fontes de dados que podem ser usados:

- Textos soltos em Python.

- PDFs.

- Arquivos .txt, .csv, .md.

- FAQs e políticas da empresa.

#### Exemplo 1: Vetorizando textos simples




In [ ]:
from langchain.text_splitter import CharacterTextSplitter

textos = """
A loja funciona de segunda a sexta-feira, das 8h às 17h.
Para trocas ou devoluções, é necessário entrar em contato em até 7 dias.
""".strip()

# Quebrar o texto em blocos
splitter = CharacterTextSplitter(separator="\n", chunk_size=100, chunk_overlap=20)
blocos = splitter.create_documents([textos])

# Vetorizando
vectorstore = FAISS.from_documents(blocos, OpenAIEmbeddings())


#### Exemplo 2: Vetorizando um arquivo .txt ou .pdf



In [ ]:
from langchain.document_loaders import TextLoader, PyPDFLoader

# Carregar texto ou PDF
loader = TextLoader("documentos/politicas.txt")  # ou PyPDFLoader("documentos/manual.pdf")
documentos = loader.load()

# Dividir e vetorizar
blocos = splitter.split_documents(documentos)
vectorstore = FAISS.from_documents(blocos, OpenAIEmbeddings())


✅ Agora temos uma base de conhecimento vetorizada a partir de arquivos reais.


## 📌 3. Buscando informações na base de vetores



In [ ]:
# Pergunta do usuário
pergunta = "Qual o horário de atendimento?"

# Transformar a pergunta em vetor e buscar similaridade
docs_relevantes = vectorstore.similarity_search(pergunta)

for doc in docs_relevantes:
    print(doc.page_content)


## 📌 Tecnologias para Armazenamento Vetorial (Banco de Dados de Vetores)

### ✅ O que é um Banco Vetorial?

É um sistema de armazenamento e recuperação que, em vez de fazer buscas por igualdade (=, LIKE), usa distância entre vetores para encontrar os itens mais semelhantes a um texto, imagem ou outro vetor.

🧠 Útil em:

- RAG (LangChain, LlamaIndex).

- Busca semântica.

- Recomendação de conteúdo.

- Sistemas de IA que precisam entender o "significado" de algo.



### 🧩 1. FAISS (Facebook AI Similarity Search)

- 💡 Tipo: Biblioteca local para busca vetorial rápida.

- 💻 Armazenamento: Em memória ou persistência simples (via pickle).

- 🏃‍♀️ Velocidade: Extremamente rápido para uso local.

- ❌ Não possui interface tipo SQL nativa.

- ✅ Ideal para: protótipos locais, projetos de pequeno/médio porte.

📦 Suportado por: LangChain, LlamaIndex, Haystack

In [ ]:
from langchain.vectorstores import FAISS
vectorstore = FAISS.from_documents(docs, OpenAIEmbeddings())


### 🧩 2. ChromaDB

- 💡 Tipo: Banco vetorial open-source, com API local e persistência em disco.

- 📦 Armazena vetores + metadados estruturados (como um SQL simplificado).

- 🧠 Suporta filtros por metadados!

- 🌐 Pode ser usado como servidor ou localmente.

- ✅ Ideal para: projetos persistentes, com múltiplos usuários e filtros.

📦 Suportado por: LangChain, LlamaIndex


In [ ]:
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(docs, OpenAIEmbeddings(), persist_directory="./db")


### 🧩 3. Weaviate

- 💡 Tipo: Banco de dados vetorial com estrutura semântica tipo GraphQL.

- 🔍 Suporte a queries estruturadas (por texto e metadados).

- 🧠 Usa filtros e contexto no estilo SQL + AI.

- **🌐 Hospedável via Docker, Cloud ou API.

- ✅ Ideal para: soluções corporativas, sistemas em produção com múltiplos tipos de dados.

📦 Suportado por: LangChain, LlamaIndex, diretamente via SDK Python

### 🧩 4. Pinecone

- 💡 Tipo: SaaS (serviço em nuvem especializado em vetores).

- 🌐 Rápido, escalável, com alta disponibilidade.

- 📘 Interface parecida com banco de dados, mas voltado para similaridade.

- 🧠 Suporta metadados e filtros.

- 💰 Plano gratuito com limitações.

📦 Suportado por: LangChain, LlamaIndex, OpenAI

In [ ]:
import pinecone
pinecone.init(api_key="SUA_CHAVE", environment="us-west1-gcp")
index = pinecone.Index("meu_index")


### 🧩 5. Milvus

- 💡 Tipo: Banco de dados vetorial open-source e altamente escalável.

- 🔧 Suporte a estrutura tipo SQL via MilvusQL.

- 📊 Pode armazenar bilhões de vetores com metadados.

- ⚙️ Requer mais setup (Docker, container).

📦 Suportado por: LangChain, Zilliz Cloud

### 🧩 6. Qdrant

- 💡 Tipo: Banco vetorial moderno, open-source e rápido.

- 🧠 Excelente suporte a filtros por metadados.

- 🌐 Acessível por API REST ou client Python.

- ✅ Interface simples + persistência.

📦 Suportado por: LangChain

![tabeladbb](tabeladb.png)

![cenarios](cenarios.png)

# Vamos usar o ChromaDB

O ChromaDB é um banco de dados vetorial open-source que armazena e busca documentos vetorizados com base em similaridade semântica.

👉 Ele funciona como um “banco de dados de significados”, em vez de palavras exatas.

Imagine que você tem uma FAQ com dezenas de perguntas/respostas, mas o usuário escreve a dúvida com outras palavras.

💡 Com o ChromaDB, você pode vetorizar essas perguntas e permitir que a IA encontre a resposta certa com base no significado, não nas palavras.



### ✅ Passo a Passo: Como funciona o ChromaDB na prática

#### 🥇 Etapa 1: Instalação
```bash
pip install chromadb langchain
```


#### 🥈 Etapa 2: O que é necessário para usar o ChromaDB com LangChain?

![cromaneed](cromaneed.png)




#### 🥉 Etapa 3: Criando o Banco Vetorial com ChromaDB


📁 Suponha que você tenha um documento de conhecimento:

In [ ]:
from langchain.docstore.document import Document

documentos = [
    Document(page_content="O prazo para reembolso é de até 7 dias úteis."),
    Document(page_content="Nosso horário de atendimento é de segunda a sexta, das 9h às 18h."),
    Document(page_content="O suporte pode ser contatado via suporte@empresa.com."),
]


🔡 Vetorizando os documentos

In [ ]:
from langchain.embeddings import OpenAIEmbeddings  # ou HuggingFaceEmbeddings
embeddings = OpenAIEmbeddings()


💾 Criando o VectorStore com Chroma

In [ ]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documentos,
    embedding=embeddings,
    persist_directory="./meu_banco_chroma"
)


📌 O Chroma agora criou uma base vetorial persistente em disco!

#### 🧪 Etapa 4: Consultando o banco com uma pergunta


Pergunta de exemplo: 

```python
pergunta = "Como faço para pedir reembolso?"
```


Buscar documentos similares:

```python
resultados = vectorstore.similarity_search(pergunta, k=2)

for doc in resultados:
    print(doc.page_content)

```

💡 O Chroma retorna os documentos mais parecidos em significado, não em palavras!



#### 🧠 Etapa 5: Integrando com um modelo de IA (RAG)

Agora que temos os documentos mais relevantes, vamos pedir a uma IA que gere uma resposta com base neles.



In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(openai_api_key="SUA_CHAVE"),
    retriever=vectorstore.as_retriever()
)

resposta = qa.run("Como posso solicitar reembolso?")
print(resposta)


🧱 Resumo de Componentes Chave do ChromaDB

![resumo](resumo.png)

## Como fazer para inserir documentos para serem vetorizados?

É possível transformar arquivos como PDF, Word, TXT e até HTML em vetores para o ChromaDB, e você pode fazer isso de forma manual via código ou usando loaders automáticos do LangChain.

LangChain tem loaders prontos para arquivos como:

![loaders](loaders.png)



### PDF → vetores no Chroma



In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# 1. Carregando o PDF
loader = PyPDFLoader("documentos/politica_reembolso.pdf")
documentos = loader.load()

# 2. Separando o conteúdo em blocos pequenos (chunking)
splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
blocos = splitter.split_documents(documentos)

# 3. Vetorizando e salvando no Chroma
vectorstore = Chroma.from_documents(
    documentos=blocos,
    embedding=OpenAIEmbeddings(),
    persist_directory="./meu_banco_vetorial"
)



## ⚖️ Comparando: Inserir via Código vs Arquivos Carregados

![comparativo](comparativo.png)



## 🚦 Quando usar cada abordagem?

![comparativo2](comparativo2.png)

### 💡 Dica Extra: Combinando os dois métodos

Você pode:

- Criar uma base com alguns textos no código.
- Acrescentar arquivos reais (PDF, CSV, etc).
- Misturar tudo antes de vetorizar.

In [ ]:
documentos_combinados = textos_do_codigo + pdfs_carregados
vectorstore = Chroma.from_documents(documentos_combinados, OpenAIEmbeddings())


## Vamos testar a vetorização de coumentos:

### 1. RAG com ChromaDB

📄 PDF fictício de uma política de reembolso da empresa "Infinity Tech".

🧠 Script Python que:

Carrega o PDF.

Divide em blocos de texto (chunking).

Vetoriza os blocos com embeddings.

Armazena tudo no ChromaDB.

🧪 Atividade prática para os alunos, com perguntas para testar o RAG.

### 2. Script Python – Carregando e Vetorizando o PDF

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# 1. Carregar o PDF
loader = PyPDFLoader("politica_reembolso_infinity_tech.pdf")
documentos = loader.load()

# 2. Dividir em blocos (chunking)
splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)
blocos = splitter.split_documents(documentos)

# 3. Vetorizar com OpenAI embeddings e salvar no ChromaDB
vectorstore = Chroma.from_documents(
    documentos=blocos,
    embedding=OpenAIEmbeddings(),  # ou HuggingFaceEmbeddings()
    persist_directory="./chroma_db_infinity"
)

print("Base vetorial criada com sucesso!")


### 3. Script de Consulta



In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(openai_api_key="SUA_API_KEY"),
    retriever=vectorstore.as_retriever()
)

pergunta = "Como posso pedir um reembolso?"
resposta = qa.run(pergunta)
print("Resposta do chatbot:", resposta)


## Como estruturar um PDF ideal para uso em RAG (Chatbot IA)

Uma das principais dores de quem trabalha com RAG é receber PDFs mal formatados, com muito ruído ou estrutura desorganizada, o que prejudica a vetorização e a recuperação de informações.

Você pode resolver isso criando um modelo-padrão de estrutura de PDF, com boas práticas e instruções claras para o cliente preencher. Vamos fazer isso agora!

📌 Objetivo:

Orientar seu cliente a montar um PDF limpo, bem dividido e estruturado, que facilite a vetorização com ferramentas como LangChain + ChromaDB.

## 📄 Modelo de Estrutura Padrão do PDF

#### 1 Use Títulos Claros e Numerados
Evite blocos gigantes de texto. Use seções com numeração.


```bash
1. SOBRE A EMPRESA
A Infinity Tech é uma empresa especializada em soluções digitais...

2. POLÍTICA DE REEMBOLSO
2.1 O cliente tem até 7 dias úteis para solicitar reembolso.
2.2 O reembolso será feito via o mesmo método de pagamento.

3. CONTATO
3.1 E-mail: suporte@infinitytech.com.br
3.2 Telefone: (11) 4000-1234
```


#### 2 Não misture temas

✅ Use 1 tema por parágrafo.
❌ Não agrupe vários assuntos no mesmo bloco.
Isso ajuda o RAG a dividir corretamente os chunks.

#### 3 Evite elementos gráficos

❌ Tabelas com muitas colunas.
❌ Imagens com informações importantes.
✅ Texto plano e estruturado é mais confiável para extração.

#### 4 Use frases completas

Evite frases curtas tipo bullet points soltos.

```bash
✅ O cliente deve enviar o comprovante por e-mail para dar início ao processo de troca.
❌ Enviar comprovante. E-mail. Troca.
```



#### 5 Dê preferência a formatos editáveis

Se possível, peça o conteúdo em .docx ou .txt para facilitar o pré-processamento.
Você pode converter para PDF posteriormente com controle.


📝 Texto de instrução para enviar ao cliente

```bash
Prezado(a),
Para que possamos integrar suas informações ao nosso sistema de IA com maior precisão e eficiência, solicitamos que o documento enviado atenda ao seguinte modelo:

Use títulos e subtítulos numerados para cada tema.

Mantenha os textos em blocos coerentes, com um tópico por parágrafo.

Evite gráficos ou tabelas complexas.

Use frases completas, evite anotações soltas.

Envie preferencialmente em .docx, .txt ou .pdf gerado a partir de texto puro (não escaneado).

Você pode seguir o modelo que enviamos como base:
(anexe o PDF de exemplo que posso gerar para você agora)